In [3]:
import pandas as pd
import random
from faker import Faker

# Initialize Faker
fake = Faker()

id_makanan = [
    'WAR01_TMH', 'WAR01_TMI', 'WAR02_TGH', 'WAR02_TGI', 'WAR03_TTH', 'WAR03_TTI',
    'WAR04_KAH', 'WAR05_ABH', 'WAR06_INH', 'WAR06_INI', 'WAR07_WCH', 'WAR07_WCI',
    'WAR08_GDH', 'WAR08_GDI', 'WAR09_BBH', 'WAR09_BBI', 'WAR10_DAH', 'WAR11_NUH',
    'WAR11_NUI', 'WAR12_KBH', 'WAR12_KBI', 'WAR13_EXH', 'WAR13_EXI', 'COF01_KTH',
    'COF02_KGA', 'COF03_CLT', 'COF04_VLT', 'COF05_CMC', 'COF06_CHH', 'COF06_CHI',
    'COF07_TTH', 'COF07_TTI', 'COF08_LTH', 'COF08_LTI', 'COF09_SSS', 'FOO01_NT',
    'FOO02_NTK', 'FOO03_NO', 'FOO04_SP', 'FOO05_IP', 'FOO06_IT', 'FOO07_ITK',
    'FOO08_IKJ', 'FOO09_IT', 'FOO10_ITD', 'FOO11_OM', 'BEN01_B1', 'BEN02_B2',
    'BEN03_B3', 'BEN04_B4', 'BEN05_BS', 'SNA01_RBC', 'SNA02_RBK', 'SNA03_RBJ',
    'SNA04_RB2', 'SNA05_RB3', 'SNA06_FF', 'SNA07_OT', 'SNA08_NU', 'SNA09_SO',
    'SNA10_MP'
]

# Define number of rows and column structure
num_rows = 1000  # Change this to the desired number of rows
columns = [
    {"name": "id_order", "type": "id_order"},
    {"name": "id_makanan", "type": "id_makanan"},
    {"name": "qty", "type": "qty"} 
]

# Generate synthetic data
synthetic_data = pd.DataFrame()

# Generate unique `id_order` and `Id_OD`
id_orders = [f"OO{str(i).zfill(4)}" for i in range(1, num_rows + 1)]

for column in columns:
    if column["type"] == "id_order":
        synthetic_data[column["name"]] = id_orders
    elif column["type"] == "id_makanan":
        # Randomly assign table IDs
        synthetic_data[column["name"]] = [random.choice(id_makanan) for _ in range(num_rows)]
    elif column["type"] == "qty":
        synthetic_data[column["name"]] = [random.randint(1, 11) for _ in range(num_rows)]
    else:
        synthetic_data[column["name"]] = [f"Data_{i}" for i in range(1, num_rows + 1)]

# Save the dataset to an Excel file
output_path = "ODetails.xlsx"
synthetic_data.to_excel(output_path, index=False)

print(f"Synthetic dataset saved to {output_path}")


Synthetic dataset saved to ODetails.xlsx


In [4]:
import pandas as pd
import mysql.connector

# Define your MariaDB connection
server = 'localhost'  # Replace with your server name
username = 'root'     # Your MariaDB username
password = ''         # Your MariaDB password
database = 'uasdb'    # The database name
table_name = 'orderdetails' # The table name in your database

# Establish connection
try:
    conn = mysql.connector.connect(
        host=server,
        user=username,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    print("Connection to the database was successful.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

# Load the dataset
input_file = "ODetails.xlsx"
synthetic_data = pd.read_excel(input_file)

# Debugging: Check column names and data
print("Columns in Excel file:", synthetic_data.columns)
print("First few rows:\n", synthetic_data.head())

# Ensure proper column mapping
synthetic_data.rename(columns={"table_id": "id_meja"}, inplace=True)

# Insert data into MariaDB
try:
    for index, row in synthetic_data.iterrows():
        cursor.execute(f"""
            INSERT INTO {table_name} (id_order, id_makanan, qty)
            VALUES (%s, %s, %s)
        """, 
        (row['id_order'], row['id_makanan'], row['qty']))

    conn.commit()
    print("Data successfully inserted into MariaDB!")
except Exception as e:
    print(f"Error inserting data: {e}")
finally:
    # Close connection
    cursor.close()
    conn.close()
    print("Database connection closed.")



Connection to the database was successful.
Columns in Excel file: Index(['id_order', 'id_makanan', 'qty'], dtype='object')
First few rows:
   id_order id_makanan  qty
0   OO0001   BEN01_B1    6
1   OO0002  COF02_KGA    3
2   OO0003  FOO10_ITD    7
3   OO0004  WAR06_INI    2
4   OO0005   FOO05_IP    9
Data successfully inserted into MariaDB!
Database connection closed.
